In [1]:
import cv2
import numpy
import os
import face_recognition
import datetime

In [2]:
alg = "haarcascade_frontalface_default.xml"
haar_cascade = cv2.CascadeClassifier(alg)

In [3]:
path = 'known_faces'
images = []
classNames = []
faceList = os.listdir(path)

for x, cl in enumerate(faceList):
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])

print(classNames)

['Bapallua', 'Nathan', 'Putra']


In [4]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

encodeListKnown = findEncodings(images)

In [8]:
test_img = cv2.imread("realmadrid.jpg")
facesCurFrame = haar_cascade.detectMultiScale(test_img, scaleFactor=1.1, minNeighbors=2, minSize=(150,150))
print(type(facesCurFrame))

<class 'numpy.ndarray'>


In [14]:
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    if not success:
        print("Failed to grab frame")
        break

    img_small = cv2.resize(img, (0, 0), fx=0.25, fy=0.25)
    
    gray = cv2.cvtColor(img_small, cv2.COLOR_BGR2GRAY)
    
    faces = haar_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=3, minSize=(10, 10))
    
    encodesCurFrame = []
    face_locations = []

    for (x, y, w, h) in faces:
        top, right, bottom, left = y, x + w, y + h, x
        face_locations.append((top, right, bottom, left))
        face_roi = img_small[top:bottom, left:right]
        face_rgb = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
        encodings = face_recognition.face_encodings(face_rgb)
        
        if encodings:
            encodesCurFrame.append(encodings[0])

    for encodeFace, faceLoc in zip(encodesCurFrame, face_locations):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        face_distances = face_recognition.face_distance(encodeListKnown, encodeFace)
        
        matchIndex = numpy.argmin(face_distances) if face_distances.size > 0 else None
        
        if matchIndex is not None and face_distances[matchIndex] < 0.50:
            name = classNames[matchIndex].upper()
        else:
            name = "Unknown"
        
        y1, x2, y2, x1 = [v * 4 for v in faceLoc]  # Scale back up to original size
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
        cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
    
    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()